# tootltips

In [8]:
!curl http://localhost:9200/dpe_pre_1948_td005/_count

{"count":0,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    71  100    71    0     0   1543      0 --:--:-- --:--:-- --:--:--  1543


# Imports

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk,parallel_bulk

# Preparer la base de données pour l'analyse au moment de la sauvegarde

In [248]:
def clean_desc_txt(x):
    bad_chars_to_space = ['"', "'", "''", '""',  # these char in addr field generate errors on addok
                      ",",  # removed by precaution since we dump comma sep csv and we dont want quotechar
                      "\\n", "\\t","\\r","\\r\\n"  #
                      "\n", "\t",  # remove carriage return/tab
                      "/", "\\", "(", ")"  # remove special separators characters and ()
                                      "[", "]", "_", "°", "^","<br>", ">", "<"
                                                                    '«', '»',

                      ]
    for bad_char in bad_chars_to_space:
        x=x.replace(bad_char,' ')
    return x

In [203]:
def setup_es_client():
    es_client = Elasticsearch()

    index_name = "dpe_extract_text"
    index_configurations = {
        "settings": {
            "index": {
                "number_of_shards": 1,
                "number_of_replicas": 0,
                "max_result_window": 5000000,

            }
        },
        "mappings": {
            "properties": {
                "id": {"type": "keyword"},
                "text_to_analyze": {
                    "type": "text",
                    #"analyzer": "french"      
                },
                "category_index":{
                    'type':'keyword'
                }
            }
        }
    }

    try:
        es_client.indices.delete(
            index=index_name,
        )
    except:
        pass
    es_client.indices.create(
        index=index_name,
        body=index_configurations
    )
    return es_client

# fonctions

In [257]:
from generate_dpe_annexes_scripts.utils import strip_accents

def gendata(index_name,id_data,data):
    for id_,vr in zip(id_data,data):
        yield {
            "_index": index_name,
    "id": id_,
    "text_to_analyze": vr
        }

def generate_instruction_from_list(char_list):
    
    new_char_list=list()
    for char in char_list:
        if isinstance(char,str):
            new_char_list.append(char)
        if isinstance(char,tuple):
            new_char_list.append('('+') OR ('.join(char)+')')
    search_instruction = '('+') AND ('.join(new_char_list) +')'
    return search_instruction
def search_from_search_dict(es_client,search_dict,index_name):
    
    s_list = list()
    for label,char_list in  search_dict.items():

        search_instruction =generate_instruction_from_list(char_list)
        search_body = {
            "query": {
                "query_string": {
                    "query": search_instruction,
                    "default_field": "text_to_analyze"
                },

            },

        }
        #es_client.count(index=index_name, body=search_body)

        a_dict=es_client.search(index=index_name, body=search_body,size=500000)

        hits = a_dict['hits']['hits']

        s=pd.Series(index=[el['_source']['id'] for el in hits])
        s[:]=label
        s_list.append(s)
    s_all=pd.concat(s_list)

    return s_all

def search_and_affect(data_to_search,id_col,val_col,search_dict,index_name='dpe_extract_text',agg_method='keep_first'):
    es_client = setup_es_client()
    bulk(es_client, gendata(index_name,data_to_search[id_col],data_to_search[val_col]))
    res_serie = search_from_search_dict(es_client,search_dict,index_name=index_name)

    res_table=res_serie.to_frame('label')

    res_table.index.name = 'id'
    res_table=res_table.reset_index()


# #     grp = df_drop.groupby('id').label.apply(lambda x: ' + '.join(sorted(list(set(x))))).reset_index()
# #     m=data.merge(grp,on='id',how='left')

#     m.label=m.label.fillna('indetermine')
    return res_table
def categorize_search_res(table,label_cat=None,category_dict=None):
    
    if category_dict is not None:
        table['category']=table.label.replace(reverse_cat_gen_ch)
    
    if label_cat is not None:
        table['label']=pd.Categorical(table.label,categories=label_cat,ordered=True)

    #df_drop = df.sort_values('label').drop_duplicates(subset=['id','category'])
    
    table.label = table.label.fillna('indetermine')
    return table

def agg_concat_label_text(table,col_label='label',col_vr='valeur_renseignee'):
    table[[col_label,col_vr]]=table[[col_label,col_vr]].fillna('indetermine')
    agg = table.groupby('td001_dpe_id').agg({

    col_label:lambda x: ' + '.join(sorted(list(set(x)))),
    col_vr:lambda x: ' + '.join(sorted(list(set(x))))
    })

    for col in agg:
        agg[col] =  agg[col].apply(lambda x:' + '.join(sorted(list(set([el for el in x.split(' + ') if el!='indetermine'])))))
        agg[col] = agg[col].replace('','indetermine')
    return agg

# load tables

In [ ]:
import generate_dpe_annexes_scripts.trtvtables
reload_package(generate_dpe_annexes_scripts.trtvtables)

In [299]:
import numpy as np

import pandas as pd
from pathlib import Path
import json
from generate_dpe_annexes_scripts.utils import round_float_cols, unique_ordered
from config import paths
from generate_dpe_annexes_scripts.trtvtables import DPETrTvTables

trtv = DPETrTvTables()

if __name__ == '__main__':

    data_dir = paths['DPE_DEPT_PATH']
    annexe_dir = paths['DPE_DEPT_ANNEXE_PATH']
    annexe_dir = Path(annexe_dir)
    annexe_dir.mkdir(exist_ok=True, parents=True)
    for dept_dir in Path(data_dir).iterdir():
        if dept_dir.name=='94':
            print(dept_dir)
            annexe_dept_dir = annexe_dir / dept_dir.name
            annexe_dept_dir.mkdir(exist_ok=True, parents=True)
            # LOAD TABLES
            td001 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
            td003 = pd.read_csv(dept_dir / 'td003_descriptif.csv', dtype=str)
            td002 = pd.read_csv(dept_dir/'td002_consommation.csv',dtype=str)
            td016 = pd.read_csv(dept_dir/'td016_facture.csv',dtype=str)

            td005 = pd.read_csv(dept_dir / 'td005_fiche_technique.csv', dtype=str)
            td006 = pd.read_csv(dept_dir / 'td006_batiment.csv', dtype=str)
            td011 = pd.read_csv(dept_dir / 'td011_installation_chauffage.csv', dtype=str)
            td012 = pd.read_csv(dept_dir / 'td012_generateur_chauffage.csv', dtype=str)

            td012_p = pd.read_csv(annexe_dept_dir / 'td012_generateur_chauffage_annexe.csv', dtype=str)
            td014_p = pd.read_csv(annexe_dept_dir / 'td014_generateur_ecs_annexe.csv', dtype=str)
            td001_sys_ch= pd.read_csv(annexe_dept_dir / 'td001_sys_ch_agg_annexe.csv', dtype=str)
            td001_sys_ecs= pd.read_csv(annexe_dept_dir / 'td001_sys_ecs_agg_annexe.csv', dtype=str)

            gen= pd.read_csv(annexe_dept_dir / 'td001_annexe_generale.csv', dtype=str)
            td001_sys_ch.columns = [col.replace('_chauffage','_ch') for col in td001_sys_ch]

            print(dept_dir)
            break
            # ENVELOPPE PROCESSING


D:\data\dpe_full\depts\94
D:\data\dpe_full\depts\94


In [319]:
td012.columns

Index(['id', 'systeme_chauffage_cogeneration_id',
       'td011_installation_chauffage_id', 'tr004_type_energie_id',
       'tv045_conversion_kwh_co2_id',
       'tv046_evaluation_contenu_co2_reseaux_id',
       'rendement_emission_systeme_chauffage',
       'tv028_rendement_emission_systeme_chauffage_id',
       'rendement_distribution_systeme_chauffage',
       'tv029_rendement_distribution_systeme_chauffage_id',
       'tv030_rendement_regulation_systeme_chauffage_id',
       'rendement_generation', 'tv031_rendement_generation_id',
       'presence_regulation', 'coefficient_performance',
       'tv032_coefficient_performance_id',
       'tv033_coefficient_correction_regulation_id',
       'tv034_temperature_fonctionnement_chaudiere_100_id',
       'tv035_temperature_fonctionnement_chaudiere_30_id', 'rpn', 'rpint',
       'qp0', 'puissance_veilleuse', 'tv036_puissance_veilleuse_id',
       'puissance_nominale', 'tv038_puissance_nominale_id',
       'consommation_chauffage'],
      dt

# dump new

In [345]:
import main
import generate_dpe_annexes_scripts.td011_td012_processing
import generate_dpe_annexes_scripts.td013_td014_processing
import generate_dpe_annexes_scripts.td001_merge
import generate_dpe_annexes_scripts.text_matching_dict

In [346]:
reload_package(main)
reload_package(generate_dpe_annexes_scripts.td011_td012_processing)
reload_package(generate_dpe_annexes_scripts.td013_td014_processing)
reload_package(generate_dpe_annexes_scripts.text_matching_dict)
reload_package(generate_dpe_annexes_scripts.td001_merge)
reload_package(main)
reload_package(generate_dpe_annexes_scripts.td011_td012_processing)
reload_package(generate_dpe_annexes_scripts.td013_td014_processing)
reload_package(generate_dpe_annexes_scripts.text_matching_dict)
reload_package(generate_dpe_annexes_scripts.td001_merge)

In [347]:
from main import *

In [348]:
td011 = pd.read_csv(dept_dir / 'td011_installation_chauffage.csv', dtype=str)
td012 = pd.read_csv(dept_dir / 'td012_generateur_chauffage.csv', dtype=str)
td013 = pd.read_csv(dept_dir / 'td013_installation_ecs.csv', dtype=str)
td014 = pd.read_csv(dept_dir / 'td014_generateur_ecs.csv', dtype=str)

td011_p, td012_p, td001_sys_ch_agg, td013_p, td014_p, td001_sys_ecs_agg = run_system_processing(td001, td006,
                                                                                                td011, td012,
                                                                                                td013, td014)
round_float_cols(td001_sys_ch_agg).to_csv(annexe_dept_dir / 'td001_sys_ch_agg_annexe.csv')
round_float_cols(td001_sys_ecs_agg).to_csv(annexe_dept_dir / 'td001_sys_ecs_agg_annexe.csv')
round_float_cols(td011_p).to_csv(annexe_dept_dir / 'td011_installation_chauffage_annexe.csv')
round_float_cols(td012_p).to_csv(annexe_dept_dir / 'td012_generateur_chauffage_annexe.csv')
round_float_cols(td013_p).to_csv(annexe_dept_dir / 'td013_installation_ecs_annexe.csv')
round_float_cols(td014_p).to_csv(annexe_dept_dir / 'td014_generateur_ecs_annexe.csv')

set()
{'tr003_nb_systeme_ch', 'tv025_I0', 'td001_dpe_id', 'tv025_type_installation', 'tv025_equipement_intermittence', 'tv025_code', 'tv025_type_batiment', 'tr003_code', 'tv025_ordre_performance', 'tv025_inertie', 'tv025_amelioration', 'tv025_type_emetteur', 'tr003_solaire', 'tr003_description', 'tv025_type_regulation'}


In [ ]:
td001_sys_ch.type_installation_ch_concat=td001_sys_ch.type_installation_ch_concat.str.replace('Chauffage Individuel',
                                                                                              'individuel').str.replace('Chauffage Collectif',
                                                                                                                        'collectif')
td001 =td001.rename(columns={'id':'td001_dpe_id'})

# td002/td016 processing

In [10]:
from generate_dpe_annexes_scripts.td002_td016_processing import merge_td002_td016_trtrv,extract_type_energie_from_td002_td016

In [51]:
td002_p,td016_p=merge_td002_td016_trtrv(td002,td016)
td001_ener_from_facture = extract_type_energie_from_td002_td016(td001,td002_p,td016_p)

# load dicts

In [252]:
from generate_dpe_annexes_scripts.utils import reload_package

In [281]:
import generate_dpe_annexes_scripts.text_matching_dict 
import os
reload_package(generate_dpe_annexes_scripts.text_matching_dict)
from generate_dpe_annexes_scripts.text_matching_dict import *

In [282]:
gen_ch_search_dict_flat.keys() -set(td012_gen_ch_search_dict.keys())

{'chaudiere energie indetermine basse temperature',
 'chaudiere energie indetermine condensation',
 'chaudiere energie indetermine indetermine',
 'chaudiere energie indetermine standard',
 'pac air/eau en releve de chaudiere',
 'pac eau/eau en releve de chaudiere',
 'pac geothermique en releve de chaudiere',
 'radiateurs gaz'}

In [283]:
set(td014_gen_ecs_search_dict.keys())-gen_ecs_search_dict_flat.keys()

{'ecs solaire thermique + ballon a accumulation electrique',
 'ecs solaire thermique + chaudiere autre : gpl butane propane basse temperature',
 'ecs solaire thermique + chaudiere autre : gpl butane propane condensation',
 'ecs solaire thermique + chaudiere autre : gpl butane propane indetermine',
 'ecs solaire thermique + chaudiere autre : gpl butane propane standard',
 'ecs solaire thermique + chaudiere bois',
 'ecs solaire thermique + chaudiere charbon basse temperature',
 'ecs solaire thermique + chaudiere charbon condensation',
 'ecs solaire thermique + chaudiere charbon indetermine',
 'ecs solaire thermique + chaudiere charbon standard',
 'ecs solaire thermique + chaudiere electrique',
 'ecs solaire thermique + chaudiere fioul basse temperature',
 'ecs solaire thermique + chaudiere fioul condensation',
 'ecs solaire thermique + chaudiere fioul indetermine',
 'ecs solaire thermique + chaudiere fioul standard',
 'ecs solaire thermique + chaudiere gaz basse temperature',
 'ecs solai

In [279]:
gen_ecs_search_dict_flat.keys() -set(td014_gen_ecs_search_dict.keys())

{'abscence ecs solaire',
 'chaudiere energie indetermine basse temperature',
 'chaudiere energie indetermine condensation',
 'chaudiere energie indetermine indetermine',
 'chaudiere energie indetermine standard',
 'chauffe-eau independant indetermine',
 'ecs solaire',
 'production mixte fioul',
 'production mixte gaz',
 'production mixte indetermine'}

# ECS

In [219]:

td005_ecs = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '17']
td005_ecs.descriptif = td005_ecs.valeur_renseignee.fillna('indetermine')

td005_ecs.valeur_renseignee = td005_ecs.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))
td005_ecs.valeur_renseignee = td005_ecs.valeur_renseignee.apply(lambda x:clean_desc_txt(x))



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [258]:

m=search_and_affect(td005_ecs,id_col='id',val_col='valeur_renseignee',
                    search_dict=gen_ecs_search_dict)

m = categorize_search_res(m,label_cat = list(gen_ecs_search_dict_flat.keys())+['indetermine'],
                           category_dict=reverse_cat_gen_ecs)
gen_ecs_lib_ft=m.merge(td005_ecs[['id','td001_dpe_id']],how='left')



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [259]:

m=search_and_affect(td003_ecs,id_col='id',val_col='descriptif',
                    search_dict=gen_ecs_search_dict)
m = categorize_search_res(m,label_cat = list(gen_ecs_search_dict_flat.keys())+['indetermine'],
                           category_dict=reverse_cat_gen_ecs)
gen_ecs_lib_desc=m.merge(td003_ecs[['id','td001_dpe_id']])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [261]:
gen_ecs_lib_from_txt=pd.concat([gen_ecs_lib_desc[['label','category','td001_dpe_id']],
                               gen_ecs_lib_ft[['label','category','td001_dpe_id']]],axis=0)

In [262]:
gen_ecs_lib_from_txt.

,label,category,td001_dpe_id
0,indetermine,solaire,5582029
1,indetermine,abscence_solaire,5582029
2,indetermine,ecs_thermodynamique,5582029
3,indetermine,solaire,5582030
4,indetermine,abscence_solaire,5582030
...,...,...,...
336938,indetermine,chauffe-eau_independant,5048002
336939,indetermine,chauffe-eau_independant,5073434
336940,indetermine,chauffe-eau_independant,5073435
336941,indetermine,chauffe-eau_independant,3955867


In [89]:

m=search_and_affect(td005_ecs,id_col='id',val_col='valeur_renseignee',
                    search_dict=gen_ecs_search_dict)

gen_ecs_lib_ft=m.merge(td005_ecs[['id','td001_dpe_id']],how='left')


m=search_and_affect(td005_ecs,id_col='id',val_col='valeur_renseignee',
                    search_dict=installation_dict)

type_installation_ecs_ft=m.merge(td005_ecs[['id','td001_dpe_id']],how='left')



m=search_and_affect(td005_ecs,id_col='id',val_col='valeur_renseignee',
                    search_dict=energie_dict,agg_method='concat')

energie_ecs_ft=m.merge(td005_ecs[['id','td001_dpe_id']],how='left')


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [90]:
td003_ecs = td003.loc[td003.tr007_type_descriptif_id == '10']
td003_ecs.descriptif = td003_ecs.descriptif.fillna('indetermine')
td003_ecs.descriptif = td003_ecs.descriptif.str.lower().apply(lambda x: strip_accents(x))
td003_ecs.descriptif = td003_ecs.descriptif.apply(lambda x:clean_desc_txt(x))



In [91]:

m=search_and_affect(td003_ecs,id_col='id',val_col='descriptif',
                    search_dict=gen_ecs_search_dict)

gen_ecs_lib_desc=m.merge(td003_ecs[['id','td001_dpe_id']])


m=search_and_affect(td003_ecs,id_col='id',val_col='descriptif',
                    search_dict=installation_dict)

type_installation_ecs_desc=m.merge(td003_ecs[['id','td001_dpe_id']])



m=search_and_affect(td003_ecs,id_col='id',val_col='descriptif',
                    search_dict=energie_dict,agg_method='concat')

energie_ecs_desc=m.merge(td003_ecs[['id','td001_dpe_id']])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


# Chauffage

In [27]:

td005_ch = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '16']
td005_ch.descriptif = td005_ch.valeur_renseignee.fillna('indetermine')

td005_ch.valeur_renseignee = td005_ch.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))
td005_ch.valeur_renseignee = td005_ch.valeur_renseignee.apply(lambda x:clean_desc_txt(x))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [79]:

m=search_and_affect(td005_ch,id_col='id',val_col='valeur_renseignee',
                    search_dict=gen_ch_search_dict_flat)

m = categorize_search_res(m,label_cat = list(gen_ch_search_dict_flat.keys())+['indetermine'],
                           category_dict=reverse_cat_gen_ch)

gen_ch_lib_ft=m.merge(td005_ch[['id','td001_dpe_id']],how='left')


m=search_and_affect(td005_ch,id_col='id',val_col='valeur_renseignee',
                    search_dict=installation_dict)

type_installation_ch_ft=m.merge(td005_ch[['id','td001_dpe_id']],how='left')



m=search_and_affect(td005_ch,id_col='id',val_col='valeur_renseignee',
                    search_dict=energie_dict,agg_method='concat')

energie_ch_ft=m.merge(td005_ch[['id','td001_dpe_id']],how='left')


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [86]:
td003_ch = td003.loc[td003.tr007_type_descriptif_id == '11']
td003_ch.descriptif = td003_ch.descriptif.fillna('indetermine')
td003_ch.descriptif = td003_ch.descriptif.str.lower().apply(lambda x: strip_accents(x))
td003_ch.descriptif = td003_ch.descriptif.apply(lambda x:clean_desc_txt(x))
  

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [87]:

m=search_and_affect(td003_ch,id_col='id',val_col='descriptif',
                    search_dict=gen_ch_search_dict_flat)

m = categorize_search_res(m,label_cat = list(gen_ch_search_dict_flat.keys())+['indetermine'],
                           category_dict=reverse_cat_gen_ch)
gen_ch_lib_desc=m.merge(td003_ch[['id','td001_dpe_id']],how='left')


m=search_and_affect(td003_ch,id_col='id',val_col='descriptif',
                    search_dict=installation_dict)

type_installation_ch_desc=m.merge(td003_ch[['id','td001_dpe_id']],how='left')



m=search_and_affect(td003_ch,id_col='id',val_col='descriptif',
                    search_dict=energie_dict,agg_method='concat')

energie_ch_desc=m.merge(td003_ch[['id','td001_dpe_id']],how='left')


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


# fusion

## méthodologie de fusion

* on priorise les informations déclarées dans le modèle de données 3CL

* on complète les données manquantes avec les données FT puis descriptif

* toutes les données indeterminées sont complétées. 


## generateurs ecs

### preparation de la donnée


In [275]:


# libéllés des generateurs chauffage issues des données textes
gen_ecs_lib_from_txt=pd.concat([gen_ecs_lib_desc[['label','category','td001_dpe_id']],
                               gen_ecs_lib_ft[['label','category','td001_dpe_id']]],axis=0)

gen_ecs_lib_from_txt =gen_ecs_lib_from_txt.sort_values(by=['td001_dpe_id','category','label'])

gen_ecs_lib_from_txt = gen_ecs_lib_from_txt.drop_duplicates(subset=['category','td001_dpe_id'],keep='first')

gen_ecs_lib_from_txt['source']='txt'

# libéllés des générateurs chauffage issus de la table td012
gen_ecs_lib_from_data = td014_p[['td001_dpe_id','gen_ecs_lib_infer']].copy()

gen_ecs_lib_from_data.columns=['td001_dpe_id','label']
gen_ecs_lib_from_data['label']=gen_ecs_lib_from_data['label'].fillna('indetermine')
gen_ecs_lib_from_data['category']=gen_ecs_lib_from_data.label.replace(reverse_cat_gen_ecs)

gen_ecs_lib_from_data['label']=pd.Categorical(gen_ecs_lib_from_data.label,categories=list(gen_ecs_search_dict_flat.keys())+['indetermine'],ordered=True)
gen_ecs_lib_from_data['label']=gen_ecs_lib_from_data['label'].fillna('indetermine')

gen_ecs_lib_from_data['source']='data'

# concat. 
gen_ecs_lib = pd.concat([gen_ecs_lib_from_data,gen_ecs_lib_from_txt],axis=0)

gen_ecs_lib['source']=pd.Categorical(gen_ecs_lib.source,categories=['data','txt'],ordered=True)
gen_ecs_lib['category']=pd.Categorical(gen_ecs_lib.category,categories=list(gen_ecs_search_dict.keys()),ordered=True)



### amélioration des labels chaudières

Problème : dans les données ou le texte on manque parfois de l'information type energie et/ou type de chaudiere (condensation/BT etc..) on cherche le maximum d'information des diverses sources pour recréer le bon libéllé

In [276]:
gen_ecs_lib_from_data

,td001_dpe_id,label,category,source
0,212,indetermine,ecs collective reseau chaleur,data
1,304,chaudiere gaz indetermine,chaudiere,data
2,566,indetermine,ecs solaire thermique + ballon a accumulation ...,data
3,898,indetermine,ecs collective reseau chaleur,data
4,1500,ballon a accumulation electrique,effet_joule,data
...,...,...,...,...
48073,9428520,ballon a accumulation electrique,effet_joule,data
48074,9428863,ballon a accumulation electrique,effet_joule,data
48075,9429496,chaudiere gaz standard,chaudiere,data
48076,9429724,ballon a accumulation electrique,effet_joule,data


In [272]:
is_chaudiere_ener_ind = gen_ecs_lib.label.str.contains('chaudiere energie indetermine')
is_chaudiere_ener_ind = is_chaudiere_ener_ind &(~gen_ecs_lib.label.str.contains('chaudiere energie indetermine indetermine'))
is_chaudiere_ind = gen_ecs_lib.label.str.contains('chaudiere')&gen_ecs_lib.label.str.contains('indetermine')&(~is_chaudiere_ener_ind)


gen_ecs_lib['chaudiere_ener_ind']=is_chaudiere_ener_ind
gen_ecs_lib['chaudiere_ind']=is_chaudiere_ind

sum_inds=gen_ecs_lib.groupby('td001_dpe_id')[['chaudiere_ener_ind','chaudiere_ind']].sum()

id_sel=sum_inds.loc[(sum_inds>0).min(axis=1)].index

ind_sel = gen_ecs_lib.td001_dpe_id.isin(id_sel)&gen_ecs_lib.chaudiere_ind
chaudiere_ind=gen_ecs_lib.loc[ind_sel]

chaudiere_ener_ind=gen_ecs_lib.loc[gen_ecs_lib.td001_dpe_id.isin(id_sel)&gen_ecs_lib.chaudiere_ener_ind]

suffix=chaudiere_ener_ind.set_index('td001_dpe_id').label.apply(lambda x:x.split('energie indetermine')[-1].strip())
suffix.name='suffix'

chaudiere_ind = chaudiere_ind.merge(suffix,on='td001_dpe_id',how='left')

chaudiere_ind.label =chaudiere_ind.label.astype(str)+ ' '+chaudiere_ind.suffix

chaudiere_ind.label=chaudiere_ind.label.str.replace('indetermine ','').str.strip()

gen_ecs_lib.loc[ind_sel,'label']=chaudiere_ind.label.values

TypeError: bad operand type for unary ~: 'float'

## generateurs chauffage


### préparation de la donnée

In [31]:
# libéllés des generateurs chauffage issues des données textes
gen_ch_lib_from_txt=pd.concat([gen_ch_lib_desc[['label','category','td001_dpe_id']],
                               gen_ch_lib_ft[['label','category','td001_dpe_id']]],axis=0)

gen_ch_lib_from_txt =gen_ch_lib_from_txt.sort_values(by=['td001_dpe_id','category','label'])

gen_ch_lib_from_txt = gen_ch_lib_from_txt.drop_duplicates(subset=['category','td001_dpe_id'],keep='first')

gen_ch_lib_from_txt['source']='txt'

# libéllés des générateurs chauffage issus de la table td012
gen_ch_lib_from_data = td012_p[['td001_dpe_id','gen_ch_lib_infer']].copy()

gen_ch_lib_from_data.columns=['td001_dpe_id','label']

gen_ch_lib_from_data['category']=gen_ch_lib_from_data.label.replace(reverse_cat_gen_ch)

gen_ch_lib_from_data['label']=pd.Categorical(gen_ch_lib_from_data.label,categories=list(gen_ch_search_dict_flat.keys())+['indetermine'],ordered=True)
gen_ch_lib_from_data['source']='data'

# concat. 
gen_ch_lib = pd.concat([gen_ch_lib_from_data,gen_ch_lib_from_txt],axis=0)

gen_ch_lib['source']=pd.Categorical(gen_ch_lib.source,categories=['data','txt'],ordered=True)
gen_ch_lib['category']=pd.Categorical(gen_ch_lib.category,categories=list(gen_ch_search_dict.keys()),ordered=True)



### amélioration des labels chaudières

Problème : dans les données ou le texte on manque parfois de l'information type energie et/ou type de chaudiere (condensation/BT etc..) on cherche le maximum d'information des diverses sources pour recréer le bon libéllé

In [34]:
is_chaudiere_ener_ind = gen_ch_lib.label.str.contains('chaudiere energie indetermine')
is_chaudiere_ener_ind = is_chaudiere_ener_ind &(~gen_ch_lib.label.str.contains('chaudiere energie indetermine indetermine'))
is_chaudiere_ind = gen_ch_lib.label.str.contains('chaudiere')&gen_ch_lib.label.str.contains('indetermine')&(~is_chaudiere_ener_ind)


gen_ch_lib['chaudiere_ener_ind']=is_chaudiere_ener_ind
gen_ch_lib['chaudiere_ind']=is_chaudiere_ind

sum_inds=gen_ch_lib.groupby('td001_dpe_id')[['chaudiere_ener_ind','chaudiere_ind']].sum()

id_sel=sum_inds.loc[(sum_inds>0).min(axis=1)].index

ind_sel = gen_ch_lib.td001_dpe_id.isin(id_sel)&gen_ch_lib.chaudiere_ind
chaudiere_ind=gen_ch_lib.loc[ind_sel]

chaudiere_ener_ind=gen_ch_lib.loc[gen_ch_lib.td001_dpe_id.isin(id_sel)&gen_ch_lib.chaudiere_ener_ind]

suffix=chaudiere_ener_ind.set_index('td001_dpe_id').label.apply(lambda x:x.split('energie indetermine')[-1].strip())
suffix.name='suffix'

chaudiere_ind = chaudiere_ind.merge(suffix,on='td001_dpe_id',how='left')

chaudiere_ind.label =chaudiere_ind.label.astype(str)+ ' '+chaudiere_ind.suffix

chaudiere_ind.label=chaudiere_ind.label.str.replace('indetermine ','').str.strip()

gen_ch_lib.loc[ind_sel,'label']=chaudiere_ind.label.values

### fusion

In [44]:
gen_ch_lib = gen_ch_lib.sort_values(by=['td001_dpe_id','category','label','source'])


In [45]:
gen_ch_lib = gen_ch_lib.sort_values(by=['td001_dpe_id','category','label','source'])

gen_ch_lib = gen_ch_lib.drop_duplicates(subset=['td001_dpe_id','category'])

gen_ch_lib=gen_ch_lib.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x)))

gen_ch_lib = gen_ch_lib.to_frame('gen_ch_lib_final')

In [46]:
td001_sys=td001.rename(columns ={'id':'td001_dpe_id'})[['td001_dpe_id']].merge(td001_sys_ch[['td001_dpe_id','gen_ch_lib_infer_concat']],on='td001_dpe_id',
                                                                              how='left')


In [47]:
td001_sys = td001_sys.merge(gen_ch_lib.reset_index(),on='td001_dpe_id',how='left')

In [48]:
diff=td001_sys.gen_ch_lib_infer_concat!=td001_sys.gen_ch_lib_final

In [49]:
ind=td001_sys.loc[td001_sys.gen_ch_lib_final=='chaudiere energie indetermine condensation']

### retraitement des energies non affectées pour les chaudières avec les energies factures

In [52]:
td001_sys =td001_sys.merge(td001_ener_from_facture,on='td001_dpe_id',how='left')


In [53]:
ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')

In [56]:
for energie in ['autre : gpl butane propane','charbon','fioul','gaz']:
    ener = td001_sys.td002_type_energie_ch.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ch_lib_final']=td001_sys.loc[ener&ind,'gen_ch_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')
    print(ind.sum())
    ener = td001_sys.td016_type_energie_ch.str.contains(energie)
    td001_sys.loc[ener&ind,'gen_ch_lib_final']=td001_sys.loc[ener&ind,'gen_ch_lib_final'].str.replace('energie indetermine',energie)
    ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('energie indetermine')
    print(ind.sum())

10204
10204
10204
10204
9807
9751
3477
3071


In [61]:
ind=td001_sys.gen_ch_lib_final.fillna('indetermine').str.contains('chaudiere energie indetermine indetermine')

elec = td001_sys.td016_type_energie_ch.str.contains('electricite')
elec = elec | td001_sys.td002_type_energie_ch.str.contains('electricite')
td001_sys.loc[ind&elec,'gen_ch_lib_final']='chaudiere electrique'

# debug

In [205]:
toto="""
systeme de production d ecs principal surface couverte = 60.0 m2 ancien ballon electrique installe en 2007. les pieces desservies sont contigues. la production est en volume chauffe. production : indiv.  ecs solaire : non  vs = 1 x 200 
"""
toto="""
 - collectif : solaire avec appoint chaudiere a condensation gaz naturel"""

In [206]:
index_name = 'dpe_extract_text'

In [207]:
search_instruction = '((reseau de chaleur) OR (reseaux de chaleurs) OR (reseaux de chaleur))'
search_instruction = generate_instruction_from_list(gen_ecs_search_dict['ecs thermodynamique electrique(PAC ou ballon)'])
search_instruction = generate_instruction_from_list(gen_ecs_search_dict['abscence ecs solaire'])

In [208]:
search_instruction = generate_instruction_from_list(gen_ecs_search_dict['ecs thermodynamique electrique(PAC ou ballon)'])


In [209]:
search_instruction

'((pompe AND chaleur) OR (pac) OR (thermodynamique) OR ("air extrait") OR ("air exterieur") OR ("air ambiant"))'

In [214]:
search_instruction ='(("ecs solaire : non") OR ("sans solaire"))'

In [215]:
#search_instruction = 'energie'

In [216]:
import time

In [217]:
search_body = {
    "query": {
        "query_string": {
            "query": search_instruction,
            "default_field": "text_to_analyze"
        },

    },

}

es_client = setup_es_client()
bulk(es_client, gendata(index_name,['id'],[toto]))

time.sleep(1)
a_dict=es_client.search(index=index_name, body=search_body,size=500000)
a_dict

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [218]:
a_dict=es_client.search(index=index_name, body=search_body,size=500000)
a_dict

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [60]:
bulk(es_client, gendata(index_name,['id'],[toto]))


(1, [])

In [53]:
es_client

<Elasticsearch([{}])>

In [54]:
a_dict

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.8630463,
  'hits': [{'_index': 'dpe_extract_text',
    '_type': '_doc',
    '_id': 'MeKchHYBLPwkNvcNz20A',
    '_score': 0.8630463,
    '_source': {'id': 'id',
     'text_to_analyze': '\nsysteme de production d ecs principal surface couverte = 60.0 m2 ancien ballon electrique installe en 2007. les pieces desservies sont contigues. la production est en volume chauffe. production : indiv.  ecs solaire : non  vs = 1 x 200 \n'}}]}}

In [ ]:

s_all = search_from_search_dict(es_client,search_dict,index_name=index_name)

In [46]:
m_ch = m

In [81]:
id_=m_ch.vr_source_ft.str.contains('air')&m_ch.vr_source_ft.str.contains('eau')&m_ch.vr_source_ft.str.contains('pac')

In [82]:
m_ch.loc[id_].to_dict(orient='records')

[{'td001_dpe_id': '1009801',
  'gen_ch_lib_ft': 'pac air/air',
  'vr_source_ft': 'installation de chauffage  283 8\xa0m²)\xa0:\r - chauffage 1 : pac air air split multisplit  energie : electrique)  avec equipement d intermittence central sans minimum de temperature  absence de regulation par piece  reseau de distribution isole',
  'type_installation_ch_ft': 'indetermine',
  'energie_ch_ft': 'Electricité non renouvelable',
  'gen_ch_lib_desc': 'pac air/air',
  'vr_source_desc': 'installation de chauffage \r - chauffage 1  pac air air split multisplit  energie electrique  reseau individuel',
  'type_installation_ch_desc': 'Chauffage Individuel',
  'energie_ch_desc': 'Chauffage Individuel',
  'gen_ch_lib_infer_concat': nan,
  'mix_energetique_ch': nan,
  'type_installation_ch_concat': nan,
  'nom_methode_dpe_norm': 'FACTURE'},
 {'td001_dpe_id': '1009804',
  'gen_ch_lib_ft': 'pac air/air',
  'vr_source_ft': 'installation de chauffage  283 8\xa0m²)\xa0:\r - chauffage 1 : pac air air split m

In [76]:
m_ch.loc[m_ch.gen_ch_lib_ft.fillna('na').str.contains('air/eau')]

,td001_dpe_id,gen_ch_lib_ft,vr_source_ft,type_installation_ch_ft,energie_ch_ft,gen_ch_lib_desc,vr_source_desc,type_installation_ch_desc,energie_ch_desc,gen_ch_lib_infer_concat,mix_energetique_ch,type_installation_ch_concat,nom_methode_dpe_norm


In [77]:
m_ch.gen_ch_lib_<Q<

,td001_dpe_id,gen_ch_lib_ft,vr_source_ft,type_installation_ch_ft,energie_ch_ft,gen_ch_lib_desc,vr_source_desc,type_installation_ch_desc,energie_ch_desc,gen_ch_lib_infer_concat,mix_energetique_ch,type_installation_ch_concat,nom_methode_dpe_norm
0,1000015,convecteurs electriques nfc,convecteur electrique nfc :\r - type de produ...,Chauffage Individuel,Electricité non renouvelable,convecteurs electriques nfc,convecteur electrique nfc\r,indetermine,indetermine,convecteurs electriques nfc,Electricité non renouvelable,Chauffage Individuel,3CL 2012
1,1000040,convecteurs electriques nfc,convecteurs electriques nfc systeme individue...,Chauffage Individuel,Electricité non renouvelable,convecteurs electriques nfc,convecteurs electriques nfc systeme individuel),Chauffage Individuel,Chauffage Individuel,autres emetteurs a effet joule,Electricité non renouvelable,Chauffage Individuel,3CL 2012
2,1000043,chaudiere fioul indetermine,chaudiere individuelle fioul installee apres 1...,Chauffage Individuel,Fioul domestique,chaudiere fioul indetermine,chaudiere individuelle fioul installee apres 1991,Chauffage Individuel,Chauffage Individuel,NaN,NaN,NaN,FACTURE
3,1000076,panneaux rayonnants electriques nfc,panneaux rayonnants nfc avec programmateur<br ...,indetermine,indetermine,panneaux rayonnants electriques nfc,panneaux rayonnants nfc avec programmateur,indetermine,indetermine,autres emetteurs a effet joule,Electricité non renouvelable,Chauffage Individuel,3CL 2012
4,1000112,chaudiere fioul indetermine,chaudiere individuelle fioul installee entre 1...,Chauffage Individuel,Fioul domestique,chaudiere fioul indetermine,chaudiere individuelle fioul installee entre 1...,Chauffage Individuel,Chauffage Individuel,NaN,NaN,NaN,FACTURE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145974,9402027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACTURE
145975,9410001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACTURE
145976,9421242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3CL 2012
145977,9421904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACTURE


In [61]:
m_ch.gen_ch_lib_ft.value_counts().to_dict()

{'reseau de chaleur': 20592,
 'chaudiere gaz indetermine': 18914,
 'indetermine': 11751,
 'chaudiere gaz basse temperature': 10978,
 'chaudiere gaz standard': 10088,
 'convecteurs electriques nfc': 9708,
 'chaudiere gaz condensation': 8251,
 'autres emetteurs a effet joule': 8158,
 'radiateurs electriques': 6936,
 'panneaux rayonnants electriques nfc': 4599,
 'chaudiere fioul indetermine': 3706,
 'chaudiere energie indetermine indetermine': 2772,
 'chaudiere fioul standard': 1348,
 'pac air/air': 1155,
 'chaudiere energie indetermine basse temperature': 647,
 'chaudiere energie indetermine condensation': 454,
 'chaudiere fioul basse temperature': 441,
 'pac eau/eau': 397,
 'plafonds/planchers rayonnants electriques nfc': 386,
 'convecteurs electriques nfc + panneaux rayonnants electriques nfc': 246,
 'poele ou insert bois': 230,
 'convecteurs electriques nfc + radiateurs electriques': 175,
 'chaudiere bois': 120,
 'pac geothermique': 116,
 'panneaux rayonnants electriques nfc + radiate

In [53]:
s=m_ch.loc[m_ch.nom_methode_dpe_norm=='FACTURE'].gen_ch_lib_ft.fillna('indetermine').value_counts()

In [57]:
s.loc['indetermine']/s.sum()

0.15338467987006182

In [55]:
(s=='indetermine').mean()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


0.0

In [35]:
m_ch.groupby('nom_methode_dpe_norm').gen_ch_lib_ft.value_counts()

nom_methode_dpe_norm  gen_ch_lib_ft                                                              
3CL 2005              convecteurs electriques nfc                                                    45
                      radiateurs electriques                                                         15
                      panneaux rayonnants electriques nfc                                            13
                      chaudiere gaz basse temperature                                                12
                      chaudiere gaz standard                                                          9
                                                                                                     ..
FACTURE               chaudiere energie indetermine indetermine + convecteurs electriques nfc         1
                      chaudiere gaz basse temperature + chaudiere gaz indetermine                     1
                      chaudiere gaz indetermine + indetermine         

In [ ]:
m.type_installation_ch_ft.value_counts()

In [ ]:
m.loc[m.gen_ch_lib_ft=='indetermine'].head(470).to_dict(orient='records')